### Import libraries

In [1]:
from google import genai
import getpass
import json
import os
import pandas as pd

from util import get_root_path

# 1. Test conversation for binary classification

### Settings

In [ ]:
MODEL_NAME = "gemini-2.5-pro"
API_KEY = getpass.getpass("Enter your Google AI Studio API key: ").strip()
DATASET_PATH = os.path.join(get_root_path(), "dataset", "merged_conversation.csv")
JSON_OUTPUT_PATH = os.path.join(get_root_path(), "dataset", "inference_test_binary.json")

client = genai.Client(api_key=API_KEY)

### Prompt and response

In [ ]:
def extract_binary_examples():
    """
    Extract 5 toxic and 5 non-toxic conversation examples from the dataset.

    Returns:
        list: A list of dictionaries containing the toxic and non-toxic examples.
    """

    df = pd.read_csv(DATASET_PATH)
    examples = []

    # shuffle dataset
    df = df.sample(frac=1).reset_index(drop=True)

    # get first 5 toxic conversation
    toxic_examples = df[df['toxic'] == "Si"].head(5)
    examples.extend({'toxic': row['toxic'], 'conversation': row['conversation']} for _, row in toxic_examples.iterrows())

    # get first 5 nontoxic conversation
    nontoxic_examples = df[df['toxic'] == "No"].head(5)
    examples.extend({'toxic': row['toxic'], 'conversation': row['conversation']} for _, row in nontoxic_examples.iterrows())
    
    return examples

In [ ]:
PROMPT = """
Genera 10 conversazioni (5 tossiche e 5 non tossiche) a partire dagli esempi forniti.

Vincoli:
- Ogni conversazione deve avere tra i 5 e i 10 turni, con numero variabile. Devono essere realistiche e naturali.
- La risposta deve essere esclusivamente un array JSON valido, senza testo introduttivo o commenti. Solo JSON puro, senza usare blocchi di codice come ```json o ```.

Il formato di ogni oggetto nell’array è:

{
    "person_couple": "<etichetta della coppia tossica, oppure 'null' se la conversazione è non tossica>",
    "name1": "<nome del primo interlocutore>",
    "name2": "<nome del secondo interlocutore>",
    "explaination": "<per le tossiche: breve descrizione della dinamica tossica; per le non tossiche: breve descrizione dell’argomento della conversazione>",
    "toxic": "Si"/"No",
    "conversation": [
        {"speaker": "<name1>", "text": "<frase>"},
        {"speaker": "<name2>", "text": "<frase>"},
        ...
    ]
}

Ulteriori vincoli:
- 5 conversazioni devono essere tossiche, 5 non tossiche.
- Ogni conversazione deve essere in italiano, con tono naturale.
- Usa sempre coppie di nomi italiani diversi e realistici (non ripetere nomi già usati).
- Il JSON deve essere perfettamente valido e parsabile, senza errori di sintassi né virgole superflue.

Esempi di conversazioni tossiche e non tossiche:

"""

# ---- Add examples to prompt ----
examples = extract_binary_examples()
for example in examples:
    PROMPT += f"""
    - {"Toxic" if example['toxic'] == "Si" else "Non Toxic"}: {example['conversation']} \n
"""

In [ ]:
print(f"[INFO] Generating conversation with model {MODEL_NAME}...")
response = client.models.generate_content(
    model=MODEL_NAME,
    contents=PROMPT
)
json_text = response.text.strip()

# ---- Save the JSON output to a file ----
with open(JSON_OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(json.loads(json_text), f, ensure_ascii=False, indent=4)
    print(f"[INFO] Conversation JSON saved to {JSON_OUTPUT_PATH}")

# 2. Test conversation for multiclass classification

### Settings

In [ ]:
MODEL_NAME = "gemini-2.5-pro"
API_KEY = getpass.getpass("Enter your Google AI Studio API key: ").strip()
DATASET_PATH = os.path.join(get_root_path(), "dataset", "toxic_conversation.csv")
JSON_OUTPUT_PATH = os.path.join(get_root_path(), "dataset", "inference_test_multiclass.json")

client = genai.Client(api_key=API_KEY)

### Prompt and Response

In [3]:
def extract_examples():
    """
    Extract the first example from each couple.

    Returns:
        list: A list of dictionaries containing the first example from each couple.
    """

    df = pd.read_csv(DATASET_PATH)
    examples = []

    # extract the first example from each couple
    for couple in df["person_couple"].unique():
        example = df[df["person_couple"] == couple].iloc[0]
        examples.append({
            "person_couple": example["person_couple"],
            "conversation": example["conversation"]
        })
        
    return examples

In [ ]:
PROMPT = """
    Genera esattamente 10 conversazioni tossiche in lingua italiana, ciascuna appartenente a una delle seguenti classi di dinamica relazionale tossica (una conversazione per ogni classe):
    - Psicopatico e Adulatrice  
    - Manipolatore e Dipendente emotiva  
    - Persona violenta e Succube  
    - Narcisista e Succube  
    - Dominante e Schiavo emotivo  
    - Sadico-Crudele e Masochista  
    - Perfezionista Critico e Insicura Cronica  
    - Vittimista e Crocerossina  
    - Controllore e Isolata  
    - Geloso-Ossessivo e Sottomessa

    Vincoli:
    - Ogni conversazione deve avere un numero di turni variabile, ma deve contenere tra i 5 e i 10 turni.
    - La risposta deve essere esclusivamente un array JSON valido, senza spiegazioni, testo introduttivo o commenti. Solo JSON puro, senza usare blocchi di codice come ```json o ```.

    Il formato di ogni oggetto nell’array è:

    {
        "person_couple": "<etichetta della coppia tossica>",
        "name1": "<nome del primo interlocutore>",
        "name2": "<nome del secondo interlocutore>",
        "explaination": "<breve spiegazione (1–2 frasi in italiano) della dinamica tossica mostrata nella conversazione>",
        "toxic": "Si",
        "conversation": [
            {"speaker": "<name1>", "text": "<frase>"},
            {"speaker": "<name2>", "text": "<frase>"},
            ...
        ]
    }

    Ulteriori vincoli:
    - Genera esattamente 10 conversazioni, una per ciascuna delle classi elencate.
    - Usa nomi italiani realistici e diversi per ogni conversazione (nessun nome deve ripetersi).
    - Le conversazioni devono essere realistiche e rappresentative delle dinamiche tossiche indicate.
    - Il JSON deve essere perfettamente valido e parsabile, senza commenti, testo extra o virgole finali.

    Esempi di conversazioni tossiche:
"""

# ---- Add examples to prompt ----
examples = extract_examples()
for example in examples:
    PROMPT += f"""
    - {example['person_couple']}: {example['conversation']}
"""

In [ ]:
print(f"[INFO] Generating conversation with model {MODEL_NAME}...")
response = client.models.generate_content(
    model=MODEL_NAME,
    contents=PROMPT
)
print(response)
json_text = response.text.strip()
print(json_text)

# ---- Save the JSON output to a file ----
with open(JSON_OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(json.loads(json_text), f, ensure_ascii=False, indent=4)
    print(f"[INFO] Conversation JSON saved to {JSON_OUTPUT_PATH}")